In [15]:
import pandas as pd

## Import data

In [16]:
df_nodes_genes = pd.read_csv('../processed_data/nodes_genes.csv')

# Edges: Protein-Gene Connection
File from: https://www.ensembl.org/biomart/martview/

Ensembl Genes 112 > Human Genes (GRCh38.p14)

Columns: Gene stable ID, Protein stable ID

In [17]:
df_edges_protein_gene = pd.read_csv('../import_data/ENSEMBLE/biomart_gene_protein.txt',  sep='\t')

# filter for genes in nodes
df_edges_protein_gene = df_edges_protein_gene[df_edges_protein_gene['Gene stable ID'].isin(df_nodes_genes['id'])]

# drop genes without protein
df_edges_protein_gene.dropna(subset=['Protein stable ID'], inplace=True)

df_edges_protein_gene

,Gene stable ID,Protein stable ID
5,ENSG00000198888,ENSP00000354687
9,ENSG00000198763,ENSP00000355046
15,ENSG00000198804,ENSP00000354499
18,ENSG00000198712,ENSP00000354876
20,ENSG00000228253,ENSP00000355265
...,...,...
185320,ENSG00000143520,ENSP00000373370
185321,ENSG00000143507,ENSP00000355866
185322,ENSG00000143507,ENSP00000483812
185323,ENSG00000143507,ENSP00000482935


# Nodes: Protein


In [18]:
df_nodes_protein = df_edges_protein_gene.copy()
df_nodes_protein = df_nodes_protein['Protein stable ID']

df_nodes_protein = df_nodes_protein.drop_duplicates()

df_nodes_protein

5         ENSP00000354687
9         ENSP00000355046
15        ENSP00000354499
18        ENSP00000354876
20        ENSP00000355265
               ...       
185320    ENSP00000373370
185321    ENSP00000355866
185322    ENSP00000483812
185323    ENSP00000482935
185324    ENSP00000480008
Name: Protein stable ID, Length: 101731, dtype: object

# Edges: Protein-Protein Interaction
STRING Data as Interaction

In [19]:
filepath = '../import_data/STRING/protein.links.full.v12.0.txt'
df_edges_protein = pd.read_csv(filepath, sep=' ')

print('Length of STRING Links Dataset: ', len(df_edges_protein))

df_edges_protein = df_edges_protein[['protein1', 'protein2']]

df_edges_protein['protein1'] = df_edges_protein['protein1'].apply(lambda x: x.split('.')[1])
df_edges_protein['protein2'] = df_edges_protein['protein2'].apply(lambda x: x.split('.')[1])

#drop duplicates
df_edges_protein = df_edges_protein.drop_duplicates()

df_edges_protein

Length of STRING Links Dataset:  13715404


,protein1,protein2
0,ENSP00000000233,ENSP00000356607
1,ENSP00000000233,ENSP00000427567
2,ENSP00000000233,ENSP00000253413
3,ENSP00000000233,ENSP00000493357
4,ENSP00000000233,ENSP00000324127
...,...,...
13715399,ENSP00000501317,ENSP00000475489
13715400,ENSP00000501317,ENSP00000370447
13715401,ENSP00000501317,ENSP00000312272
13715402,ENSP00000501317,ENSP00000402092


### Filter for Proteins from Nodes

In [20]:
df_edges_protein = df_edges_protein[df_edges_protein['protein1'].isin(df_nodes_protein)]
df_edges_protein = df_edges_protein[df_edges_protein['protein2'].isin(df_nodes_protein)]

df_edges_protein

,protein1,protein2
0,ENSP00000000233,ENSP00000356607
1,ENSP00000000233,ENSP00000427567
2,ENSP00000000233,ENSP00000253413
3,ENSP00000000233,ENSP00000493357
4,ENSP00000000233,ENSP00000324127
...,...,...
13715039,ENSP00000501277,ENSP00000424474
13715040,ENSP00000501277,ENSP00000254846
13715041,ENSP00000501277,ENSP00000352069
13715042,ENSP00000501277,ENSP00000327650


FROM LATEX:

not every protein is connected to another one 

→ but since every gene could have multiple proteins, the connected gene may have a connection to another protein

TODO: are those genes that are not connected to another gene relevant for the analysis?

In [21]:
# TODO find genes that are not connected to another gene



# Export Data

In [22]:
df_nodes_protein.to_csv('../processed_data/nodes_protein.csv', index=False)

df_edges_protein.to_csv('../processed_data/edges_protein.csv', index=False)

df_edges_protein_gene.to_csv('../processed_data/edges_protein_gene.csv', index=False)

In [23]:
print(f"There are {len(df_nodes_protein)} Protein Nodes")
print(f"There are {len(df_edges_protein)} Protein-Protein Edges")
print(f"There are {len(df_edges_protein_gene)} Protein-Gene Edges")

There are 101731 Protein Nodes
There are 11247242 Protein-Protein Edges
There are 101731 Protein-Gene Edges
